In [1]:
import pandas as pd
import numpy as np
import geoenrich
from geoenrich import dataloader, enrichment, satellite, exports
import pathlib

In [2]:
geo_buff = 115
time_buff = (0, 0)

In [6]:
enrichment.enrich(dataset_ref = 'plankton', var_id = 'nh4_med', geo_buff = geo_buff, time_buff = time_buff, slice = (0, 1000))

186679 occurrences were loaded from enrichment file
Ignoring 646 rows because data is not available at these dates


100%|██████████| 354/354 [00:10<00:00, 35.18it/s]


Enrichment over


In [2]:
df = pd.read_csv('../plankton_geoenrich_data/biodiv/plankton.csv')

In [3]:
variables = [
    'sst',
    'salinity3d'
    'oxygen', 
    'oxygen2', 
    'chlorophyll',
    'phosphate',
    # 'nitrate',
    # 'carbon-dissolved-inorganic',
]

In [ ]:
# output = exports.retrieve_data("plankton", 3897, 'oxygen', shape = 'buffer')
# values = exports.export_to_array(output)

In [3]:
exports.collate_npy(ds_ref = 'plankton_med', data_path = './npy', slice = (0, 100))

  0%|          | 0/100 [00:00<?, ?it/s]


Read error in netCDF file. If local file, try restoring a backup [{'min': nan, 'max': nan, 'step': 1}, {'min': nan, 'max': nan, 'step': 1}, {'min': nan, 'max': nan, 'step': 1}, {'min': nan, 'max': nan, 'step': 1}]


TypeError: slice indices must be integers or None or have an __index__ method

In [17]:
# Path object : 

data_path = pathlib.Path('../plankton_geoenrich_data/biodiv/plankton.csv')

In [19]:
data_path + '/' + 'plankton.csv'

TypeError: unsupported operand type(s) for +: 'PosixPath' and 'str'